In [55]:
first_file_path = "../Fajlovi/1000G_test.vcf"
second_file_path = "../Fajlovi/1000G_test_2.vcf"

merged_file = open("merged.vcf","w+")

#možda da napravim jednu veliku klasu VCF_file u kojoj ću imati metode za sortiranje dictionaries-a filters i formats

class File_format: 
    def __init__(self, line):
        self.line = line
        self.vcf_version = ""
        self.extract_file_format_version()
        
    def __eq__(self, other):
        return self.line == other.line
    
    def extract_file_format_version(self):
        try:
            version = self.line.split("=VCFv",1)[1]
            self.vcf_version = version
        except:
            pass
    

class Filter_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.description = ""
        self.extract_id()
        self.extract_description()
        
    def extract_id(self):
        try:
            id = (self.line.split("ID="))[1].split(",")[0]
            self.id = id
        except:
            pass
    
    def extract_description(self):
        try:
            description = (self.line.split('Description="'))[1].split('"')[0]
            self.description = description
        except:
            pass

# ##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
class Format_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.number = ""
        self.type = ""
        self.description = ""
        self.extract_id()
        self.extract_number()
        self.extract_type()
        self.extract_description()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_number(self):
        try:
            self.number = (self.line.split("Number="))[1].split(",")[0]            
        except:
            pass
        
    def extract_type(self):
        try:
            self.type = (self.line.split("Type="))[1].split(",")[0]            
        except:
            pass
    
    def extract_description(self):
        try:
            self.description = self.line.split('Description="', 1)[1]
        except:
            pass
        
        
def write_filter_fields(filters):
    for key, value in sorted(filters.items()):
        merged_file.write(value.line)
    
    #do I need to close merged_file ???
    #get rid of filters, GarbageCollector or just filters = {}

def write_format_fields(formats):
    for key, value in sorted(formats.items()):
        merged_file.write(value.line)
        
    
def extract_info_from_line(line):
    pass
    
    
#pošto se filteri trebaju sortirati trebam kupiti filtere u jednu listu 
#da li ovdje koristiti listu ili nešto drugo, uzećemo dictionary
#kod filtera sortiranje ide po ID 
filters = {}
formats = {}
filters_written = False
formats_written = False

#da li napraviti da je izlazni fajl kopija većeg fajla i onda dodavati dodatne linije




# context manager
with open(first_file_path) as first_file_object:
    with open(second_file_path) as second_file_object:
        for line in first_file_object:
            if(line.startswith("##fileformat")):
                file_format_first_file = File_format(line)
                first_line_in_second_file = second_file_object.readline()
                file_format_second_file = File_format(first_line_in_second_file) 
                if(file_format_first_file.vcf_version == file_format_second_file.vcf_version):
                    pass
                    #print("Isti format fajla je u oba VCF dokumenta")
                else:
                    pass
                    #print("Različit format fajlova u ova dva VCF dokumenta.")
                
            
            # ##FILTER=<ID="id",Description="description">    
            elif (line.startswith("##FILTER")):
                filter_header = Filter_header(line) 
                filters[filter_header.id] = filter_header
        
            else:
                if(filters_written == False):
                    # u prvom fajlu više nema filter linija 
                    #sad idemo u drugi fajl
                
                    previous_position = second_file_object.tell()
                    line_second_file = second_file_object.readline() 
                    while(line_second_file.startswith("##FILTER")):
                        filterHeader = FilterHeader(line_second_file) 
                        filters[filterHeader.id] = filterHeader
                        previous_position = second_file_object.tell()
                        line_second_file = second_file_object.readline()
                    
                    #rewind file 
                    second_file_object.seek(previous_position)
                    #ovo će da izađe kada line_second_file više ne bude počinjala sa ##FILTER
                    #write filter fields in output file 
                    write_filter_fields(filters)
                    filters_written = True
                
                else:
                    if(line.startswith("##FORMAT")):
                        format_header = Format_header(line) 
                        formats[format_header.id] = format_header
                    
                    else: 
                        if(formats_written == False):
                        # u prvom fajlu više nema format linija 
                        #sad idemo u drugi fajl
                
                            previous_position = second_file_object.tell()
                            line_second_file = second_file_object.readline() 
                            while(line_second_file.startswith("##FORMAT")):
                                format_header = Format_header(line_second_file) 
                                formats[format_header.id] = format_header
                            
                                previous_position = second_file_object.tell()
                                line_second_file = second_file_object.readline()
                    
                            #rewind file 
                            second_file_object.seek(previous_position)
                            #ovo će da izađe kada line_second_file više ne bude počinjala sa ##FILTER
                            #write filter fields in output file 
                            write_format_fields(formats)
                            formats_written = True
                    
    